# Causal-Embeddings Training Notebook

В этом ноутбуке реализован конвейер для обучения каузальных эмбеддингов на датасете пар (причина, следствие) с использованием PyTorch и BECAUSE 2.0.

In [1]:
# Импорт необходимых библиотек
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import random
import sklearn.metrics as M
import wandb # Import W&B
import torch.nn.functional as F # Often needed
import matplotlib.pyplot as plt

In [ ]:
# Гиперпараметры обучения
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01 # L2 регуляризация
DROPOUT_PROB = 0.2 # вероятность dropout
# MARGIN = 0.5    # margin для контрастивного loss (No longer needed for InfoNCE)
TEMPERATURE = 0.05 # шкала для InfoNCE loss
NUM_EPOCHS = 20   # Increased epochs, early stopping will handle the actual number
# STEP_SIZE = 3    # шаг для scheduler (No longer needed for CosineAnnealingLR)
# GAMMA = 0.5     # множитель для scheduler (No longer needed for CosineAnnealingLR)
BATCH_SIZE = 128   # размер батча
EARLY_STOPPING_PATIENCE = 2 # Epochs to wait for improvement before stopping

In [4]:
# Флаг: использовать предобученные эмбеддинги GPT2 или инициализировать их с нуля
use_pretrained_embeds = True # False — для random init

In [5]:
# Загрузка датасета e-CARE через HuggingFace datasets
print("Загрузка датасета 12ml/e-CARE...")
ec_dataset = load_dataset("12ml/e-CARE", split="train")
print(f"Пример записи: {ec_dataset[0]}")

Загрузка датасета 12ml/e-CARE...
Пример записи: {'idx': 'train-0', 'premise': 'There is a light rain today.', 'question': 'effect', 'choice1': 'The roots of many plants are not moistened by rain.', 'choice2': 'Tourists have seen many ripples.', 'label': 0, 'conceptual_explanation': 'Precipitation reaches soil surfaces.'}


In [6]:
# Инициализация токенизатора и модели
model_name = 'gpt2'
print(f"Используемая модель: {model_name}")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
if use_pretrained_embeds:
  base_model = GPT2Model.from_pretrained(model_name)
else:
  base_model = GPT2Model(GPT2Config())

import torch.nn.functional as F
class CausalEmbeddingModel(torch.nn.Module):
  def __init__(self, base_model, dropout_prob=DROPOUT_PROB):
    super().__init__()
    self.encoder = base_model
    self.dropout = torch.nn.Dropout(dropout_prob)

  def forward(self, input_ids, attention_mask):
    outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden = outputs.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    summed = torch.sum(last_hidden * mask, 1)
    counts = torch.clamp(mask.sum(1), min=1e-9)
    embeddings = summed / counts
    embeddings = self.dropout(embeddings)
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings

device = torch.device("mps" if torch.mps.is_available() else "cpu")
model = CausalEmbeddingModel(base_model, dropout_prob=DROPOUT_PROB).to(device)

Используемая модель: gpt2


In [7]:
# InfoNCE Loss Function
class InfoNCELoss(torch.nn.Module):
  def __init__(self, temperature=TEMPERATURE):
    super().__init__()
    self.temperature = temperature
    self.criterion = torch.nn.CrossEntropyLoss()

  def forward(self, emb1, emb2, labels):
    # emb1: [batch_size, embed_dim] (e.g., cause embeddings)
    # emb2: [batch_size, embed_dim] (e.g., effect embeddings)
    # labels: [batch_size] (1 for positive pair, 0 for negative)

    # Normalize embeddings (already done in model, but good practice)
    emb1 = F.normalize(emb1, p=2, dim=1)
    emb2 = F.normalize(emb2, p=2, dim=1)

    # Calculate cosine similarity matrix (batch_size x batch_size)
    # sim_matrix[i, j] = similarity between emb1_i and emb2_j
    sim_matrix = torch.matmul(emb1, emb2.T) / self.temperature

    # Create labels for cross-entropy: diagonal elements are positives
    # We only calculate loss for the actual positive pairs in the input `labels`
    positive_mask = (labels == 1)
    if not positive_mask.any(): # Handle batches with no positive pairs if they occur
       return torch.tensor(0.0, device=emb1.device, requires_grad=True) # Or handle as needed

    # Select rows corresponding to positive causes
    pos_sim_matrix = sim_matrix[positive_mask]

    # Create target labels for these rows: the diagonal element (true effect) should be the target
    # The index of the positive sample for emb1_i is i
    # Need to find the original indices of the positive samples
    positive_indices = torch.where(positive_mask)[0]
    # The target label for the i-th positive sample (original index k=positive_indices[i])
    # should correspond to the column k in the full similarity matrix.
    # However, CrossEntropyLoss expects class indices relative to the input tensor (pos_sim_matrix).
    # We need to map the global positive index `k` to the row index `i` within the `pos_sim_matrix`.
    # This mapping is simply the range(number_of_positives).
    num_positives = pos_sim_matrix.size(0)
    targets = torch.arange(num_positives, device=emb1.device) # Target is the index of the positive pair within the filtered matrix

    # Adjust targets based on the actual structure if emb1[i] positive is emb2[i]
    # In our case, the positive pair for emb1[k] is emb2[k].
    # So, the target column index in sim_matrix[k, :] is k.
    # When we filter sim_matrix with positive_mask, the columns remain the same.
    # We need the column indices corresponding to the positive pairs.
    targets = positive_indices # The target column index is the original index of the positive pair

    # Filter the similarity matrix columns to only include the relevant comparisons for the positive samples
    # This is complex. Let's simplify: Use the standard CE loss where the target is the index of the positive sample.
    # For emb1_i (where label_i=1), the positive is emb2_i.
    # The logits are sim(emb1_i, emb2_j) for all j. Target is i.

    # Recalculate using the standard approach for simplicity:
    # Treat each emb1 as an anchor, its corresponding emb2 as positive (if label=1)
    # and all other emb2 in the batch as negatives.

    # Cosine similarity between corresponding pairs (potential positives)
    sim_pos = torch.diag(sim_matrix) # Similarity between emb1_i and emb2_i

    # Create logits for CrossEntropyLoss
    # For each emb1_i, the logits are [sim(emb1_i, emb2_0), sim(emb1_i, emb2_1), ..., sim(emb1_i, emb2_{N-1})]
    logits = sim_matrix

    # Create targets: for row i, the target class is i (representing emb2_i)
    targets = torch.arange(emb1.size(0), device=emb1.device)

    # Calculate loss only for the positive pairs
    loss = self.criterion(logits[positive_mask], targets[positive_mask])

    return loss

criterion = InfoNCELoss(temperature=TEMPERATURE) # Use the new loss

In [8]:
# Формируем список троек (причина, следствие, label) из ec_dataset с hard negatives
pairs = []
# Собираем все возможные следствия для hard negatives
all_effects = [item['choice1'] for item in ec_dataset] + [item['choice2'] for item in ec_dataset]
for item in ec_dataset:
  cause = item['premise']
  # effect_pos — правильное следствие из поля choice1/choice2 по метке label
  effect_pos = item['choice1'] if item['label'] == 0 else item['choice2']
  # effect_neg — неправильный выбор
  effect_neg = item['choice2'] if item['label'] == 0 else item['choice1']
  # Положительная пара
  pairs.append((cause, effect_pos, 1))
  # Негативная пара (контрастивная)
  pairs.append((cause, effect_neg, 0))
  # Hard negative: случайный эффект, не равный правильному
  hn = random.choice([eff for eff in all_effects if eff != effect_pos])
  pairs.append((cause, hn, 0))

print(f"Всего пар: {len(pairs)}")
print('Пример положительной пары:', pairs[0])
print('Пример отрицательной пары:', pairs[1])
print('Количество положительных пар:', sum(1 for x in pairs if x[2] == 1))
print('Количество отрицательных пар:', sum(1 for x in pairs if x[2] == 0))

Всего пар: 44784
Пример положительной пары: ('There is a light rain today.', 'The roots of many plants are not moistened by rain.', 1)
Пример отрицательной пары: ('There is a light rain today.', 'Tourists have seen many ripples.', 0)
Количество положительных пар: 14928
Количество отрицательных пар: 29856


In [8]:
%wandb login

In [ ]:
# Initialize Weights & Biases
wandb.init(
  project="causal-embeddings-ecare",
  config={
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "dropout_prob": DROPOUT_PROB,
    "temperature": TEMPERATURE,
    "epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "model_name": model_name,
    "use_pretrained": use_pretrained_embeds,
    "scheduler": "CosineAnnealingLR",
    "loss": "InfoNCE",
    "early_stopping_patience": EARLY_STOPPING_PATIENCE
  }
)
# Log code for reproducibility
wandb.save('causal_embeddings.ipynb')

In [16]:
# Делим на train/eval
train_data, eval_data = train_test_split(pairs, test_size=0.2, random_state=42)

class LabeledCausalDataset(Dataset):
  def __init__(self, data, tokenizer, max_length=64):
    self.data = data
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    cause, effect, label = self.data[idx]
    if self.tokenizer.pad_token is None:
      self.tokenizer.pad_token = self.tokenizer.eos_token
    enc1 = self.tokenizer(cause, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
    enc2 = self.tokenizer(effect, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
    return {
      'input_ids1': enc1['input_ids'].squeeze(),
      'attention_mask1': enc1['attention_mask'].squeeze(),
      'input_ids2': enc2['input_ids'].squeeze(),
      'attention_mask2': enc2['attention_mask'].squeeze(),
      'label': torch.tensor(label, dtype=torch.float),
      'cause_text': cause,
      'effect_text': effect
    }

train_dataset = LabeledCausalDataset(train_data, tokenizer)
eval_dataset = LabeledCausalDataset(eval_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-7)

In [ ]:
# Цикл обучения с контрастивной функцией потерь и запись метрик
train_losses = [] # Store epoch average train losses
eval_aucs = []  # Store epoch evaluation AUCs
best_eval_auc = 0.0
epochs_no_improve = 0

print("Starting training...")
for epoch in range(NUM_EPOCHS):
  model.train()
  total_loss = 0
  batch_losses = [] # Track losses within an epoch
  
  for batch_idx, batch in enumerate(train_loader):
    optimizer.zero_grad()
    emb1 = model(batch['input_ids1'].to(device), batch['attention_mask1'].to(device))
    emb2 = model(batch['input_ids2'].to(device), batch['attention_mask2'].to(device))
    labels = batch['label'].to(device)
    
    # Ensure there are positive samples for InfoNCE calculation in the batch
    if (labels == 1).sum() == 0:
      print(f"Skipping batch {batch_idx+1} due to no positive samples.")
      continue # Skip batch if no positive pairs exist

    loss = criterion(emb1, emb2, labels)

    # Check for NaN loss
    if torch.isnan(loss):
      print(f"NaN loss detected at Epoch {epoch+1}, Batch {batch_idx+1}. Skipping batch.")
      # Potentially log problematic batch data here if needed
      wandb.log({"problem_batch_data": wandb.Table(data=[batch['cause_text'], batch['effect_text'], batch['label'].tolist()])})
      continue # Skip optimizer step and loss accumulation
    
    loss.backward()
    # Gradient clipping (optional but often helpful)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    
    current_loss = loss.item()
    total_loss += current_loss
    batch_losses.append(current_loss)
    
    # Log batch loss and learning rate to W&B
    current_lr = optimizer.param_groups[0]['lr']
    wandb.log({
      "batch_loss": current_loss,
      "learning_rate": current_lr,
      "epoch": epoch + (batch_idx + 1) / len(train_loader) # Log fractional epoch
    })
    
    if (batch_idx + 1) % 50 == 0: # Print progress every 50 batches
      print(f"Epoch {epoch+1} Batch {batch_idx+1}/{len(train_loader)} - Loss {current_loss:.4f}, LR: {current_lr:.2e}")

  avg_loss = total_loss / len(batch_losses) if batch_losses else 0 # Avoid division by zero
  train_losses.append(avg_loss) # Append average loss for the epoch

  # Evaluation step
  model.eval()
  y_true, y_scores = [], []
  with torch.no_grad():
    for batch in eval_loader:
      emb1 = model(batch['input_ids1'].to(device), batch['attention_mask1'].to(device))
      emb2 = model(batch['input_ids2'].to(device), batch['attention_mask2'].to(device))
      sim = F.cosine_similarity(emb1, emb2, dim=1).cpu().numpy()
      y_scores.extend(sim.tolist())
      y_true.extend(batch['label'].numpy().tolist())
  
  # Ensure evaluation data is valid before calculating AUC
  if len(np.unique(y_true)) > 1: # Check if there's more than one class present
    auc = M.roc_auc_score(y_true, y_scores)
  else:
    print(f"Warning: Only one class present in evaluation labels for Epoch {epoch+1}. AUC cannot be calculated.")
    auc = 0.0 # Or handle as appropriate, e.g., np.nan

  eval_aucs.append(auc) # Append AUC for the epoch
  
  # Log epoch metrics to W&B
  wandb.log({
    "epoch": epoch + 1, # Log integer epoch for epoch-level metrics
    "train_loss_epoch": avg_loss,
    "eval_auc_epoch": auc
  })
  
  print(f"Epoch {epoch+1}, Avg Train Loss: {avg_loss:.4f}, Eval AUC: {auc:.4f}")
  
  # Early Stopping Check
  if auc > best_eval_auc:
    best_eval_auc = auc
    epochs_no_improve = 0
    # Save the best model checkpoint (optional)
    best_model_path = os.path.join(wandb.run.dir, 'best_model.pth') # Save in W&B run dir
    torch.save(model.state_dict(), best_model_path)
    print(f"New best model saved with AUC: {best_eval_auc:.4f}")
    wandb.save(best_model_path) # Save best model to W&B
  else:
    epochs_no_improve += 1
    print(f"No improvement in Eval AUC for {epochs_no_improve} epoch(s).")

  if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
    print(f"Early stopping triggered after {epoch + 1} epochs.")
    break # Exit training loop

  # Step the scheduler (CosineAnnealingLR steps per epoch)
  scheduler.step()

print("Training finished.")
# Log final best AUC
wandb.summary["best_eval_auc"] = best_eval_auc

In [ ]:
# Финальная оценка на eval наборе с cosine similarity
model.eval()
y_true, y_scores = [], []
with torch.no_grad():
  for batch in eval_loader:
    emb1 = model(batch['input_ids1'].to(device), batch['attention_mask1'].to(device))
    emb2 = model(batch['input_ids2'].to(device), batch['attention_mask2'].to(device))
    sim = torch.nn.functional.cosine_similarity(emb1, emb2, dim=1).cpu().numpy()
    y_scores.extend(sim.tolist())
    y_true.extend(batch['label'].numpy().tolist())
auc = M.roc_auc_score(y_true, y_scores)
print(f"Eval AUC after e-CARE training: {auc:.4f}")
wandb.summary["final_eval_auc_ecare"] = auc # Log final e-CARE AUC to the first run

In [ ]:
# Сохраняем обученные эмбеддинги causal в ../models
os.makedirs(os.path.join(os.getcwd(), '..', 'models'), exist_ok=True)
embeds = model.encoder.wte.weight.data.cpu()
torch.save(embeds, os.path.join(os.getcwd(), '..', 'models', 'causal_embeds.pth'))
print("Causal embeddings saved to ../models/causal_embeds.pth")

#### Фаза 2: Дообучение на Atomic и CNC

Теперь продолжим обучение модели, используя данные из датасетов Atomic и CNC. Мы будем использовать те же веса модели, которые были получены после обучения на e-CARE.

In [13]:
# Оптимизированное формирование atomic_pairs
print("Загрузка и обработка датасета Atomic...")
atomic_df = pd.read_csv('../datasets/atomic_causal_pairs.csv')
atomic_df.dropna(subset=['cause', 'effect'], inplace=True)

cause_to_effects = atomic_df.groupby('cause')['effect'].apply(set).to_dict()
all_atomic_effects_set = set(atomic_df['effect'].unique())
print(f"Найдено уникальных причин в Atomic: {len(cause_to_effects)}")
print(f"Найдено уникальных следствий в Atomic: {len(all_atomic_effects_set)}")

# Эффективное формирование позитивных и негативных пар
atomic_pairs = []
for cause, effects in cause_to_effects.items():
    # Положительные пары
    for effect_pos in effects:
        atomic_pairs.append((cause, effect_pos, 1))
    # Подготовка кандидатов для негативных примеров
    total_negs = len(effects) * 2
    neg_cands = list(all_atomic_effects_set - effects)
    if len(neg_cands) >= total_negs:
        neg_samples = random.sample(neg_cands, total_negs)
    else:
        fallback = neg_cands if neg_cands else list(all_atomic_effects_set)
        neg_samples = random.choices(fallback, k=total_negs)
    # Назначение по два негативных эффекта на каждую позитивную пару
    for idx in range(len(effects)):
        for neg_effect in neg_samples[2*idx:2*idx+2]:
            atomic_pairs.append((cause, neg_effect, 0))

print(f"Сформировано пар из Atomic: {len(atomic_pairs)}")
print('Пример положительной пары (Atomic):', next((p for p in atomic_pairs if p[2] == 1), None))
print('Пример отрицательной пары (Atomic):', next((p for p in atomic_pairs if p[2] == 0), None))
print('Количество положительных пар (Atomic):', sum(1 for x in atomic_pairs if x[2] == 1))
print('Количество отрицательных пар (Atomic):', sum(1 for x in atomic_pairs if x[2] == 0))

Загрузка и обработка датасета Atomic...
Найдено уникальных причин в Atomic: 23667
Найдено уникальных следствий в Atomic: 88334
Сформировано пар из Atomic: 402180
Пример положительной пары (Atomic): ('Aaron accidentally tripped', 'cries from pain', 1)
Пример отрицательной пары (Atomic): ('Aaron accidentally tripped', 'gets excited to receive invitation', 0)
Количество положительных пар (Atomic): 134060
Количество отрицательных пар (Atomic): 268120


In [14]:
# Загрузка и обработка датасета CNC
print("\nЗагрузка и обработка датасета CNC...")
cnc_df = pd.read_csv('../datasets/cnc_causal_pairs.csv')
cnc_df.dropna(subset=['cause', 'effect', 'is_causal'], inplace=True)
cnc_df = cnc_df[cnc_df['is_causal'] == 1]

cause_to_effects_cnc = cnc_df.groupby('cause')['effect'].apply(set).to_dict()
all_cnc_effects_set = set(cnc_df['effect'].unique())
print(f"Найдено уникальных причин в CNC: {len(cause_to_effects_cnc)}")
print(f"Найдено уникальных следствий в CNC: {len(all_cnc_effects_set)}")

cnc_pairs = []
for cause, effects in cause_to_effects_cnc.items():
    # Положительные пары
    for effect_pos in effects:
        cnc_pairs.append((cause, effect_pos, 1))
    # Подготовка кандидатов для негативных примеров (2 на каждый позитив)
    total_negs = len(effects) * 2
    neg_cands = list(all_cnc_effects_set - effects)
    if len(neg_cands) >= total_negs:
        neg_samples = random.sample(neg_cands, total_negs)
    else:
        fallback = neg_cands if neg_cands else list(all_cnc_effects_set)
        neg_samples = random.choices(fallback, k=total_negs)
    # Добавляем негативные пары по два на каждую позитивную
    for idx in range(len(effects)):
        for neg_effect in neg_samples[2*idx:2*idx+2]:
            cnc_pairs.append((cause, neg_effect, 0))

print(f"Сформировано пар из CNC: {len(cnc_pairs)}")
print('Пример положительной пары (CNC):', next((p for p in cnc_pairs if p[2] == 1), None))
print('Пример отрицательной пары (CNC):', next((p for p in cnc_pairs if p[2] == 0), None))
print('Количество положительных пар (CNC):', sum(1 for x in cnc_pairs if x[2] == 1))
print('Количество отрицательных пар (CNC):', sum(1 for x in cnc_pairs if x[2] == 0))


Загрузка и обработка датасета CNC...
Найдено уникальных причин в CNC: 2423
Найдено уникальных следствий в CNC: 2468
Сформировано пар из CNC: 7512
Пример положительной пары (CNC): (' Accusing the CPM of instigating the violence', 'she said the deaths were result of clashes between the Congress and the Marxists', 1)
Пример отрицательной пары (CNC): (' Accusing the CPM of instigating the violence', 'The Supreme Court Tuesday asked the Gujarat government to pay Rs.1 million in ex-gratia to the mother and three brothers of Sohrabuddin Sheikh', 0)
Количество положительных пар (CNC): 2504
Количество отрицательных пар (CNC): 5008


In [17]:
# Объединение данных и создание DataLoader'ов для Фазы 2
print("\nОбъединение данных и подготовка DataLoader'ов для Фазы 2...")
combined_pairs_phase2 = atomic_pairs + cnc_pairs
random.shuffle(combined_pairs_phase2) # Перемешиваем объединенный датасет

print(f"Всего пар для Фазы 2: {len(combined_pairs_phase2)}")
print('Количество положительных пар (Фаза 2):', sum(1 for x in combined_pairs_phase2 if x[2] == 1))
print('Количество отрицательных пар (Фаза 2):', sum(1 for x in combined_pairs_phase2 if x[2] == 0))

# Делим на train/eval для Фазы 2
train_data_phase2, eval_data_phase2 = train_test_split(combined_pairs_phase2, test_size=0.15, random_state=43) # Используем другое соотношение и random_state

print(f"Размер обучающей выборки (Фаза 2): {len(train_data_phase2)}")
print(f"Размер валидационной выборки (Фаза 2): {len(eval_data_phase2)}")

# Создаем Dataset и DataLoader
# Используем тот же класс LabeledCausalDataset и тот же токенизатор
train_dataset_phase2 = LabeledCausalDataset(train_data_phase2, tokenizer)
eval_dataset_phase2 = LabeledCausalDataset(eval_data_phase2, tokenizer)

train_loader_phase2 = DataLoader(train_dataset_phase2, batch_size=BATCH_SIZE, shuffle=True)
eval_loader_phase2 = DataLoader(eval_dataset_phase2, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoader'ы для Фазы 2 готовы.")


Объединение данных и подготовка DataLoader'ов для Фазы 2...
Всего пар для Фазы 2: 409692
Количество положительных пар (Фаза 2): 136564
Количество отрицательных пар (Фаза 2): 273128
Размер обучающей выборки (Фаза 2): 348238
Размер валидационной выборки (Фаза 2): 61454
DataLoader'ы для Фазы 2 готовы.


In [ ]:
# Инициализация нового W&B run для Фазы 2
# Завершаем предыдущий run, если он еще активен (на всякий случай)
if wandb.run is not None:
    print("Завершение предыдущего W&B run...")
    try:
        wandb.finish()
    except Exception as e:
        print(f"Error finishing previous W&B run: {e}. Continuing...")

print("\\nИнициализация W&B для Фазы 2 обучения (Atomic + CNC)...")
wandb.init(
  project="causal-embeddings-atomic-cnc", # Новое имя проекта или группы
  config={
    "learning_rate": LEARNING_RATE / 5, # Можно уменьшить LR для дообучения # CHANGED: Lowered LR further
    "weight_decay": WEIGHT_DECAY,
    "dropout_prob": DROPOUT_PROB,
    "temperature": TEMPERATURE,
    "epochs": NUM_EPOCHS, # Используем то же макс. количество эпох
    "batch_size": BATCH_SIZE,
    "model_name": model_name,
    "use_pretrained": use_pretrained_embeds, # Указывает на исходную инициализацию
    "training_phase": 2, # Добавляем флаг фазы
    "scheduler": "CosineAnnealingLR",
    "loss": "InfoNCE",
    "early_stopping_patience": EARLY_STOPPING_PATIENCE
  }
)

# Log code for reproducibility
wandb.save('causal_embeddings.ipynb')

# Переинициализация оптимизатора и планировщика для Фазы 2
# Используем текущие параметры модели, но с возможно измененным LR
optimizer_phase2 = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE / 5, weight_decay=WEIGHT_DECAY) # CHANGED: Lowered LR further
scheduler_phase2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_phase2, T_max=NUM_EPOCHS, eta_min=1e-8) # Можно установить eta_min еще ниже

# Сброс переменных для early stopping и метрик
train_losses_phase2 = [] 
eval_aucs_phase2 = []  
best_eval_auc_phase2 = 0.0 # Начинаем с нуля для этой фазы
epochs_no_improve_phase2 = 0

# Используем тот же criterion
# criterion = InfoNCELoss(temperature=TEMPERATURE) # Уже определен ранее

print("Оптимизатор, планировщик и переменные для Фазы 2 инициализированы.")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


\nИнициализация W&B для Фазы 2 обучения (Atomic + CNC)...


wandb: Currently logged in as: vladotpad (vladsteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Оптимизатор, планировщик и переменные для Фазы 2 инициализированы.


In [20]:
# Цикл обучения Фазы 2 (Atomic + CNC)
print("\nStarting training Phase 2 (Atomic + CNC)...")
# Убедимся, что модель в режиме обучения
model.to(device) # На всякий случай перемещаем модель на нужное устройство
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss_phase2 = 0
    batch_losses_phase2 = [] 
  
    for batch_idx, batch in enumerate(train_loader_phase2):
        optimizer_phase2.zero_grad()
        
        # Перемещаем данные батча на устройство
        input_ids1 = batch['input_ids1'].to(device)
        attention_mask1 = batch['attention_mask1'].to(device)
        input_ids2 = batch['input_ids2'].to(device)
        attention_mask2 = batch['attention_mask2'].to(device)
        labels = batch['label'].to(device)

        emb1 = model(input_ids1, attention_mask1)
        emb2 = model(input_ids2, attention_mask2)
        
        # Ensure there are positive samples for InfoNCE calculation in the batch
        if (labels == 1).sum() == 0:
            print(f"Phase 2 - Skipping batch {batch_idx+1} due to no positive samples.")
            continue # Skip batch if no positive pairs exist

        loss = criterion(emb1, emb2, labels)
        
        # Check for NaN loss
        if torch.isnan(loss):
            print(f"Phase 2 - NaN loss detected at Epoch {epoch+1}, Batch {batch_idx+1}. Skipping batch.")
            # Log problematic batch data
            try:
                wandb.log({"phase2_problem_batch_data": wandb.Table(data=[batch['cause_text'], batch['effect_text'], batch['label'].tolist()])})
            except Exception as e:
                print(f"Could not log problematic batch to W&B: {e}")
            continue 
        
        print(f"Batch {batch_idx+1}")
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer_phase2.step()
    
    current_loss = loss.item()
    total_loss_phase2 += current_loss
    batch_losses_phase2.append(current_loss)
    
    # Log batch loss and learning rate to W&B (Phase 2)
    current_lr_phase2 = optimizer_phase2.param_groups[0]['lr']
    wandb.log({
      "phase2_batch_loss": current_loss,
      "phase2_learning_rate": current_lr_phase2,
      "phase2_epoch_frac": epoch + (batch_idx + 1) / len(train_loader_phase2) # Log fractional epoch for phase 2
    })
    
    if (batch_idx + 1) % 100 == 0: # Print progress every 100 batches
        print(f"Phase 2 - Epoch {epoch+1} Batch {batch_idx+1}/{len(train_loader_phase2)} - Loss {current_loss:.4f}, LR: {current_lr_phase2:.2e}")

    avg_loss_phase2 = total_loss_phase2 / len(batch_losses_phase2) if batch_losses_phase2 else 0 
    train_losses_phase2.append(avg_loss_phase2) 

    # Evaluation step (Phase 2)
    model.eval()
    y_true_phase2, y_scores_phase2 = [], []
    with torch.no_grad():
        for batch in eval_loader_phase2:
            # Перемещаем данные батча на устройство
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)
            
            emb1 = model(input_ids1, attention_mask1)
            emb2 = model(input_ids2, attention_mask2)
            sim = F.cosine_similarity(emb1, emb2, dim=1).cpu().numpy()
            y_scores_phase2.extend(sim.tolist())
            y_true_phase2.extend(batch['label'].numpy().tolist())

    # Ensure evaluation data is valid before calculating AUC
    if len(np.unique(y_true_phase2)) > 1: 
        auc_phase2 = M.roc_auc_score(y_true_phase2, y_scores_phase2)
    else:
        print(f"Phase 2 - Warning: Only one class present in evaluation labels for Epoch {epoch+1}. AUC cannot be calculated.")
        auc_phase2 = 0.0 

    eval_aucs_phase2.append(auc_phase2) 

    # Log epoch metrics to W&B (Phase 2)
    wandb.log({
        "phase2_epoch": epoch + 1, 
        "phase2_train_loss_epoch": avg_loss_phase2,
        "phase2_eval_auc_epoch": auc_phase2
    })

    print(f"Phase 2 - Epoch {epoch+1}, Avg Train Loss: {avg_loss_phase2:.4f}, Eval AUC: {auc_phase2:.4f}")
  
    # Early Stopping Check (Phase 2)
    if auc_phase2 > best_eval_auc_phase2:
        best_eval_auc_phase2 = auc_phase2
        epochs_no_improve_phase2 = 0
        # Save the best model checkpoint for phase 2
        best_model_path_phase2 = os.path.join(wandb.run.dir, 'best_model_phase2.pth') 
        torch.save(model.state_dict(), best_model_path_phase2)
        print(f"Phase 2 - New best model saved with AUC: {best_eval_auc_phase2:.4f}")
        wandb.save(best_model_path_phase2) # Save best model to W&B
    else:
        epochs_no_improve_phase2 += 1
        print(f"Phase 2 - No improvement in Eval AUC for {epochs_no_improve_phase2} epoch(s).")

    if epochs_no_improve_phase2 >= EARLY_STOPPING_PATIENCE:
        print(f"Phase 2 - Early stopping triggered after {epoch + 1} epochs.")
        break # Exit training loop

    # Step the scheduler (Phase 2)
    scheduler_phase2.step()

print("Training Phase 2 finished.")
# Log final best AUC for Phase 2
wandb.summary["phase2_best_eval_auc"] = best_eval_auc_phase2


Starting training Phase 2 (Atomic + CNC)...
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106
Bat

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Phase 2 - New best model saved with AUC: 0.4781
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x3023e0e30>> (for post_run_cell), with arguments args (<ExecutionResult object at 3317aee10, execution_count=20 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 3317aede0, raw_cell="# Цикл обучения Фазы 2 (Atomic + CNC)
print("\nSta.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/vladislavkalinichenko/VSCodeProjects/SMILES_my_research/implementation/notebooks/causal_embeddings.ipynb#X25sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# Финальная оценка на eval наборе Фазы 2
print("\nFinal Evaluation on Phase 2 Eval Set...")
model.eval()
y_true_final_p2, y_scores_final_p2 = [], []
with torch.no_grad():
  for batch in eval_loader_phase2:
    # Перемещаем данные батча на устройство
    input_ids1 = batch['input_ids1'].to(device)
    attention_mask1 = batch['attention_mask1'].to(device)
    input_ids2 = batch['input_ids2'].to(device)
    attention_mask2 = batch['attention_mask2'].to(device)
      
    emb1 = model(input_ids1, attention_mask1)
    emb2 = model(input_ids2, attention_mask2)
    sim = torch.nn.functional.cosine_similarity(emb1, emb2, dim=1).cpu().numpy()
    y_scores_final_p2.extend(sim.tolist())
    y_true_final_p2.extend(batch['label'].numpy().tolist())

if len(np.unique(y_true_final_p2)) > 1:
    final_auc_p2 = M.roc_auc_score(y_true_final_p2, y_scores_final_p2)
    print(f"Final Eval AUC (Phase 2): {final_auc_p2:.4f}")
    wandb.summary["final_eval_auc_phase2"] = final_auc_p2
else:
    print("Final Eval AUC (Phase 2): Could not be calculated (only one class in eval set).")
    wandb.summary["final_eval_auc_phase2"] = None

In [ ]:
# Сохраняем финальные эмбеддинги после Фазы 2
print("\nSaving final causal embeddings after Phase 2...")
final_embeds_path = os.path.join(os.getcwd(), '..', 'models', 'causal_embeds_final_atomic_cnc.pth')
os.makedirs(os.path.dirname(final_embeds_path), exist_ok=True)
embeds_final = model.encoder.wte.weight.data.cpu()
torch.save(embeds_final, final_embeds_path)
print(f"Final causal embeddings saved to {final_embeds_path}")
wandb.save(final_embeds_path) # Save final embeddings to W&B

In [ ]:

# Визуализация результатов Фазы 2
print("\nPlotting Phase 2 training curves...")
# Determine the actual number of epochs run in Phase 2
actual_epochs_phase2 = len(train_losses_phase2) 

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
if actual_epochs_phase2 > 0:
    plt.plot(range(1, actual_epochs_phase2 + 1), train_losses_phase2, '-o')
plt.title('Phase 2: Average Train Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
if actual_epochs_phase2 > 0:
    plt.plot(range(1, actual_epochs_phase2 + 1), eval_aucs_phase2, '-o')
plt.title('Phase 2: Evaluation AUC per Epoch')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.grid(True)

plt.tight_layout()
plt.show()

# Log the final plot to W&B (Phase 2 run)
try:
    wandb.log({"phase2_training_curves": plt}) 
except Exception as e:
    print(f"Could not log plot to W&B: {e}")

# Finish the W&B run for Phase 1 here
print("Finishing W&B run for Phase 1...")
try:
    wandb.finish()
    print("Phase 1 W&B run finished.")
except Exception as e:
    print(f"Error finishing Phase 1 W&B run: {e}. Continuing...")

In [ ]:
os.makedirs(os.path.join(os.getcwd(), '..', 'models'), exist_ok=True)
embeds = model.encoder.wte.weight.data.cpu()
torch.save(embeds, os.path.join(os.getcwd(), '..', 'models', 'causal_embeds.pth'))
print("Causal embeddings saved to ../models/causal_embeds.pth")


## Как происходит дообучение эмбеддингов GPT-2

В этом ноутбуке дообучение эмбеддингов GPT-2 происходит с помощью контрастивного обучения:

- Для каждой пары (причина, следствие, label) тексты токенизируются с помощью GPT2Tokenizer и пропускаются через GPT2Model.
- Полученные эмбеддинги (среднее по токенам) используются как представления причины и следствия.
- Контрастивная функция потерь (ContrastiveLoss) минимизирует расстояние между эмбеддингами для позитивных пар (реальных причинно-следственных связей) и максимизирует для негативных (альтернативных) пар.
- Градиенты распространяются только через параметры GPT2Model (в том числе эмбедтинговый слой), что приводит к дообучению эмбеддингов под задачу каузального различения.
- После обучения эмбеддинги GPT-2 становятся более чувствительными к каузальным отношениям, что можно использовать для интеграции в downstream-задачи или модификации внимания.

# Далее: улучшение сходимости
- Перейти на InfoNCE (NT-Xent) loss: формировать батчи с несколькими негативами, вычислять матрицу similarity и применять кросс-энтропию.
- Добавить projection head (MLP) поверх эмбеддингов:  GPT2 эмбеддинг → Linear → ReLU → Linear, затем L2-нормализовать.
- Использовать более агрессивный lr scheduler (CosineAnnealingLR, ReduceLROnPlateau) и раннюю остановку.
- Поэкспериментировать с batch size и количеством negative samples на одну positive пару.
- Увеличить разнообразие hard negatives: например, брать эффекты из соседних записей на основе семантической близости.

In [ ]:
# Determine the actual number of epochs run
actual_epochs = len(train_losses) # Use the length of the recorded losses/aucs

# Plot training loss and evaluation AUC
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, actual_epochs + 1), train_losses, '-o')
plt.title('Average Train Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, actual_epochs + 1), eval_aucs, '-o')
plt.title('Evaluation AUC per Epoch')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.grid(True)

plt.tight_layout()
plt.show()

# Log the final plot to W&B
wandb.log({"training_curves": plt}) # Log the matplotlib figure object

# Finish the W&B run for Phase 1 here
print("Finishing W&B run for Phase 1...")
try:
    wandb.finish()
    print("Phase 1 W&B run finished.")
except Exception as e:
    print(f"Error finishing Phase 1 W&B run: {e}. Continuing...")